In [20]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:red">ch2. Ollama_LLM활용의기본개념(LangChain)</span>
# 1. LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM 이용
성능은 GPT, Claude 같은 모델보다 떨어지나, 개념설명을 위해 open source 모델 사용

### ollama.com 다운로드 -> 설치 -> 모델 pull
- cmd창이나 powershell 창에 ollama pull deepseek-r1:1.5b
- https://docs.langchain.com/oss/python/integrations/chat/ollama

In [3]:
from langchain_ollama import ChatOllama
lim = ChatOllama(model="deepseek-r1:1.5b")
result = lim.invoke("What is the capital of korea?")
result

AIMessage(content='The capital of Korea, South Korea, is首尔 (Gyeongju).', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-09T02:05:28.0991212Z', 'done': True, 'done_reason': 'stop', 'total_duration': 820767300, 'load_duration': 87335400, 'prompt_eval_count': 11, 'prompt_eval_duration': 74415200, 'eval_count': 22, 'eval_duration': 555465700, 'logprobs': None, 'model_name': 'deepseek-r1:1.5b', 'model_provider': 'ollama'}, id='lc_run--019b00db-3f68-7010-8e23-aac03c8eb481-0', usage_metadata={'input_tokens': 11, 'output_tokens': 22, 'total_tokens': 33})

### 모델 pull
- cmd창이나 powershell창(window키+R에서 powershell)에서 ollama pull llama3.2:1b

In [4]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model='llama3.2:1b')
result = llm.invoke("What is the capital of korea?")
result

AIMessage(content='The capital of South Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T02:18:44.2244317Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1829748600, 'load_duration': 1259168500, 'prompt_eval_count': 33, 'prompt_eval_duration': 292793800, 'eval_count': 9, 'eval_duration': 196534800, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b00e7-6159-7b63-8af7-e8884a48b0e1-0', usage_metadata={'input_tokens': 33, 'output_tokens': 9, 'total_tokens': 42})

In [5]:
result.content

'The capital of South Korea is Seoul.'

In [6]:
result = llm.invoke("한국 수도가 어디에요?")
result.content

'한국의 수도는 Seoul입니다.'

## 2) openAI 활용
- pip install langchain-openai
- https://auth.openai.com/log-in

In [10]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# os.getenv("OPENAI_API_KEY")

True

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano",
                 # api_key=os.getenv("OPENAI_API_KEY")
                )
result = llm.invoke("What is the capital of korea? Return the name of the city only.")
result

AIMessage(content='Seoul', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 203, 'prompt_tokens': 21, 'total_tokens': 224, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CkkVnOQu9UCcvfuxQ0QHy3qXFqSz5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0192-223d-7651-96ba-8918925747c4-0', usage_metadata={'input_tokens': 21, 'output_tokens': 203, 'total_tokens': 224, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [13]:
# Azure : OPENAI_API_VERSION 키값
# from langchain_openai import AzureChatOpenAI
# llm = AzureChatOpenAI(model="gpt-5-nano")

# 2. 랭체인 스타일로 프롬프트 작성
- 프롬프트 : llm호출시 쓰는 질문

In [22]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model='llama3.2:1b')
# llm.invoke(0)
# PromptValue, str, BaseMessages 리스트

## 1) 기본 프롬프트 템플릿 사용
- PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [23]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model='llama3.2:1b')
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)
# country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":"korea"})
print(prompt)
llm.invoke(prompt)

text='What is the capital of korea?'


AIMessage(content='The capital of Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:21:03.1163968Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1796647400, 'load_duration': 1292417300, 'prompt_eval_count': 33, 'prompt_eval_duration': 258639400, 'eval_count': 8, 'eval_duration': 225950100, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01c5-3a06-72e2-b465-92016517cd8a-0', usage_metadata={'input_tokens': 33, 'output_tokens': 8, 'total_tokens': 41})

## 2) 메세지 기반 프롬프트 작성
- BaseMessage 리스트
- BaseMessage 상속받은 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage
- vscode에서 ctrl+shift+p : python:select interpreter입력 -> python 환경 선택
- vscode에서 커널 선택

In [21]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범 질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범 답안
    HumanMessage(content="What is the capital of France?"), # 모범 질문
    AIMessage(content="The capital of Italy is Paris."),    # 모범 답안
    HumanMessage(content="What is the capital of korea?")
]
llm.invoke(message_list)

AIMessage(content='The capital of South Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:07:13.527022Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2363354400, 'load_duration': 1296962200, 'prompt_eval_count': 87, 'prompt_eval_duration': 778038700, 'eval_count': 9, 'eval_duration': 262655400, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01b8-8f39-76c1-a7a1-848a723cc38d-0', usage_metadata={'input_tokens': 87, 'output_tokens': 9, 'total_tokens': 96})

In [24]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범 질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범 답안
    HumanMessage(content="What is the capital of France?"), # 모범 질문
    AIMessage(content="The capital of Italy is Paris."),    # 모범 답안
    HumanMessage(content="What is the capital of {country}?")
]
llm.invoke(message_list)

AIMessage(content="If you're referring to a specific country, I'd be happy to help. Please specify the name of the country, and I'll do my best to provide the correct capital.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:21:11.2974054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1972007400, 'load_duration': 110547800, 'prompt_eval_count': 87, 'prompt_eval_duration': 588827200, 'eval_count': 37, 'eval_duration': 1112868000, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01c5-594b-7741-ad91-1901fc917106-0', usage_metadata={'input_tokens': 87, 'output_tokens': 37, 'total_tokens': 124})

## 3) ChatPromptTemplate 사용
- BaseMessage리스트->튜플리스트

In [27]:
# 위의 BaseMessage리스트를 수정
# PromptTemplate : 프롬프트에 변수 포함
# ChatPromptTemplate : SystemPrompt설정(페르소나), few shot설정, 변수포함
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant!"),
    ("human", "What is the capital of Italy?"), # 모범 질문
    ("ai", "The capital of Italy is Rome."),    # 모범 답안
    ("human", "What is the capital of France?"), # 모범 질문
    ("ai", "The capital of Italy is Paris."),    # 모범 답안
    ("human", "What is the capital of {country}?")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPrompt_template.invoke({"country": country})
# print("프롬프트 : ", prompt, type(prompt))
result = llm.invoke(prompt)
result.content

어느 나라 수도가 궁금하세요korea


'The capital of South Korea is Seoul, and the capital of North Korea is Pyongyang.'

In [31]:
chatPromptTemplate = ChatPromptTemplate.from_messages([
    ('system', "당신은 대한민국 정보 전문 도우미입니다"),
    ('human', "{country}의 수도가 어디에요!")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPromptTemplate.invoke({'country':country})
# print(prompt)
result = llm.invoke(prompt)
print(result.content)

어느 나라 수도가 궁금하세요korea
 korea의 수도는 Seoul입니다!


In [32]:
result

AIMessage(content=' korea의 수도는 Seoul입니다!', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:39:41.0494634Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2003045100, 'load_duration': 1377801900, 'prompt_eval_count': 44, 'prompt_eval_duration': 349438900, 'eval_count': 9, 'eval_duration': 242851300, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d6-4824-7ea3-a329-377579419669-0', usage_metadata={'input_tokens': 44, 'output_tokens': 9, 'total_tokens': 53})

# 3. 답변 형식 컨트롤 하기
- llm.invoke()의 결과는 AIMessage() -> string이나 json, 객체 : OutputParser이용

## 1) 문자열 출력 파서 이용
- StrOutputParser를 사용하여 LLM출력(AIMessage)을 단순 문자열로 변환

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Return the name of the city only.",
    input_variables = ["country"]
)
# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country":"korea"})
# print(prompt)
ai_message = llm.invoke(prompt)
# print(ai_message)
# 문자열 출력 파서를 이용하요 llm응답(AIMessage)을 단순 문자열로 변환
output_parser = StrOutputParser()
result = output_parser.invoke(ai_message)
result

'Seoul'

In [36]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"korea"})))

'Seoul'

In [39]:
# 변수설정, system, few shot 지정
chat_prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant with expertise in south korea."),
    ("human", "What is the capital of Italy?"), # 모범 질문
    ("ai", "Rome."),    # 모범 답안
    ("human", "What is the capital of France?"), # 모범 질문
    ("ai", "Paris."),    # 모범 답안
    ("human", "What is the capital of {country}? Return the name of the city only")
])
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country":"korea"})))

'Seoul.'

## 2) Json 출력 파서 이용
- {'name' : '홍길동', 'age' : 22}

In [59]:
from langchain_core.output_parsers import JsonOutputParser
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in is JSON format and return the JSON dictionary only""",
    input_variables=['country']
)
prompt = country_detail_prompt.invoke({"country":"korea"})
ai_message = llm.invoke(prompt)
# print(ai_message.content)
output_parser = JsonOutputParser()
json_result = output_parser.invoke(ai_message)
print(json_result, type(json_result))

{'capital': 'Seoul', 'population': 51.83, 'language': 'Korean', 'currency': 'Won'} <class 'dict'>


In [56]:
output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country":"japan"})))

{'capital': 'Tokyo',
 'population': 128843811,
 'language': 'Japanese',
 'currency': ' Yen'}

## 3) 구조화된 출력 사용
- Pydantic 모델을 사용하여 LLM출력을 구조화된 형식으로 받기(JsonParser보다 훨씬 안정적)
- Pydantic : 데이터유효성검사, 설정관리를 간편하게 해주는 라이브러리

In [63]:
class User:
    def __init__(self, id, name, is_active=True):
        self.id = id
        self.name = name,
        self.is_active = is_active
user = User("1", "홍길동", False)
print(user)

In [67]:
from pydantic import BaseModel, Field
class User(BaseModel):
    # gt=0:id>=0 / ge=0:id>=0 / lt=0:id<0 / ie=0:id<=0
    id:int = Field(gt=0, description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id활성화 여부")
user = User(id="1", name="홍길동")
print(user)

id=1 name='홍길동' is_active=True


In [68]:
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in is JSON format and return the JSON dictionary only""",
    input_variables=['country']
)
class CountryDetail(BaseModel): # description : 더 정확한 출력 유도
    capital:str = Field(description="the Capital of the country")
    population:int = Field(description="the Population of the country")
    language:str = Field(description="the Language of the country")
    currency:str = Field(description="the Currency of the country")
# 출력 형식 파서 + LLM
structedllm = llm.with_structured_output(CountryDetail)
# llm.invoke(country_detail_prompt.invoke({"country":"korea"}))
info = structedllm.invoke(country_detail_prompt.invoke({"country":"korea"}))
info

CountryDetail(capital='Seoul', population=51, language='Korean', currency='South Korean won')

In [69]:
type(info)

__main__.CountryDetail

In [70]:
info.capital, info.population, info.language, info.currency

('Seoul', 51, 'Korean', 'South Korean won')

In [72]:
print("info를 json으로 :", info.model_dump_json())
print("info를 dict로 :", info.model_dump())
print("info를 dict로 :", info.__dict__)

info를 json으로 : {"capital":"Seoul","population":51,"language":"Korean","currency":"South Korean won"}
info를 dict로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'South Korean won'}
info를 dict로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'South Korean won'}


In [73]:
type(info.model_dump_json())

str

# 4. LCEL을 활용한 랭체인 생성하기
## 1) 문자열 출력 파서 사용
- invoke : Runnable에 있는 함수
- StrOutputParser, ChatOllama, PromptTemplate등은 모두 Runnable로 상속 받음.

In [78]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) LCEL을 사용한 간단한 체인 구성
- 파이프연산자(|) 이용

In [79]:
# 프롬프트 템플릿 -> llm -> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"korea"})

'Seoul'

## 3) 복합체인 구성
- 여러 단계의 추론이 필요한 경우(체인연결)

In [81]:
# 나라설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following informat:
    {information}
    Return the name of the country only""",
    input_variables=['information']
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":"This country is very famous for its wine"})))

'Italy'

In [82]:
# 나라명 추출 체인 생성
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information":"This country is very famous for its wine"})

'Italy'

In [83]:
# 복합체인 : 나라설명 -> 나라명(country_chain)
#                     나라명 -> 수도(capital_chain)
final_chain = country_chain | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [84]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | \
                 {"country":country_chain} | capital_chain

In [86]:
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

- 한글 지원이 안되는 모델은 랭체인 연결이 잘 안 됨

In [88]:
# 나라설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""다음의 {information} 설명을 보고 나라이름을 맞춰봐:
    {information}
    나라 이름만 한국어로 return 해 줘""",
    input_variables=['information']
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":"이 나라는 와인으로 유명해"})))

'이 나라는 와인으로 유명해\n\n1. 프랑스 - burgundy, merlot, cabernet sauvignon\n2. 이탈리아 - chianti, pinot noir, vermentino\n3. 스타일러 - merlot, shiraz, syrah\n4. 이스라엘 - merlot, cabernet sauvignon, pinot noir\n5. 미국 - merlot, cabernet sauvignon, pinot noir'